In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
# tokenize

def tokenize(text):
  text = text.lower()
  text = text.replace('?', '')
  text = text.replace("'","")
  return text.split()


In [5]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [6]:
# vocab
vocab = {'<UNK>':0}

In [8]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)


In [9]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [10]:
len(vocab)

324

In [12]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [13]:
text_to_indices('What is campusx', vocab)

[1, 2, 0]

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

In [15]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)


In [16]:
dataset = QADataset(df, vocab)

In [17]:
dataloader  =  DataLoader(dataset, batch_size=1, shuffle=True)

In [18]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([321])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([154])
tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([121])
tensor([[ 10,  75, 208]]) tensor([209])
tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]]) tensor([52])
tensor([[  1,   2,   3,   4,   5, 236, 237]]) tensor([238])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([7])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([136])
tensor([[  1,   2,   3,  69,   5, 155]]) tensor([156])
tensor([[ 42, 290, 291, 118, 292, 158, 293, 294]]) tensor([295])
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]]) tensor([61])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([54])
tensor([[ 1,  2,  3,  4,  5, 99]]) tensor([100])
tensor([[  1,   2,   3,  17, 115,  83,  84]]) tensor([116])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([114])
tensor([[ 42, 167,   2,   3,  17, 168, 169]]) tensor([170])
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]]) tensor([173])
tensor([[10, 75, 76]]) tensor([77])


In [19]:
import torch.nn as nn

In [20]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim = 50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [21]:
x = nn.Embedding(324, embedding_dim = 50)
y = nn.RNN(50, 64, batch_first= True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print('shape of a:', a.shape)
b = x(a)
print('shape of b:', b.shape)
c,d = y(b)
print('shape of c:', c.shape)
print('shape of d:', d.shape)

e = z(d.squeeze(0))

print('shape of e:', e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [22]:
learning_rate  = 0.001
epochs = 20

In [23]:
model = SimpleRNN(len(vocab))

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [25]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output  = model(question)

    # loss -> output shape (1, 324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f'Epoch: {epoch+1}, Loss: {total_loss:4f}')

Epoch: 1, Loss: 525.371914
Epoch: 2, Loss: 458.729573
Epoch: 3, Loss: 378.699100
Epoch: 4, Loss: 316.184889
Epoch: 5, Loss: 264.927125
Epoch: 6, Loss: 217.569415
Epoch: 7, Loss: 173.866728
Epoch: 8, Loss: 135.346182
Epoch: 9, Loss: 103.828936
Epoch: 10, Loss: 79.631220
Epoch: 11, Loss: 61.839338
Epoch: 12, Loss: 48.034098
Epoch: 13, Loss: 38.286477
Epoch: 14, Loss: 30.819312
Epoch: 15, Loss: 25.338982
Epoch: 16, Loss: 21.303743
Epoch: 17, Loss: 17.999361
Epoch: 18, Loss: 15.200991
Epoch: 19, Loss: 13.010206
Epoch: 20, Loss: 11.232573


In [26]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")
  print(list(vocab.keys())[index])

In [27]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [28]:
list(vocab.keys())[7]

'paris'